In [12]:
import re, random, math, collections, itertools

In [1]:
def readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia):

    #reading pre-labeled input and splitting into lines
    posSentences = open('rt-polarity.pos', 'r', encoding="ISO-8859-1")
    posSentences = re.split(r'\n', posSentences.read())

    negSentences = open('rt-polarity.neg', 'r', encoding="ISO-8859-1")
    negSentences = re.split(r'\n', negSentences.read())

    posSentencesNokia = open('nokia-pos.txt', 'r')
    posSentencesNokia = re.split(r'\n', posSentencesNokia.read())

    negSentencesNokia = open('nokia-neg.txt', 'r', encoding="ISO-8859-1")
    negSentencesNokia = re.split(r'\n', negSentencesNokia.read())
 
    posDictionary = open('positive-words.txt', 'r', encoding="ISO-8859-1")
    posWordList = re.findall(r"[a-z\-]+", posDictionary.read())

    negDictionary = open('negative-words.txt', 'r', encoding="ISO-8859-1")
    negWordList = re.findall(r"[a-z\-]+", negDictionary.read())

    for i in posWordList:
        sentimentDictionary[i] = 1
    for i in negWordList:
        sentimentDictionary[i] = -1
 
    for i in posSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="positive"
        else:
            sentencesTrain[i]="positive"

    for i in negSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="negative"
        else:
            sentencesTrain[i]="negative"

    #create Nokia Datset:
    for i in posSentencesNokia:
            sentencesNokia[i]="positive"
    for i in negSentencesNokia:
            sentencesNokia[i]="negative"

In [14]:
#calculates p(W|Positive), p(W|Negative) and p(W) for all words in training data


def trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord):
    posFeatures = [] # [] initialises a list [array]
    negFeatures = [] 
    freqPositive = {} # {} initialises a dictionary [hash function]
    freqNegative = {}
    dictionary = {}
    posWordsTot = 0
    negWordsTot = 0
    allWordsTot = 0

    #iterate through each sentence/sentiment pair in the training data
    for sentence, sentiment in sentencesTrain.items():
        wordList = re.findall(r"[\w']+", sentence)
        
        for word in wordList: #calculate over unigrams
            allWordsTot += 1 # keeps count of total words in dataset
            if not (word in dictionary):
                dictionary[word] = 1
            if sentiment=="positive" :
                posWordsTot += 1 # keeps count of total words in positive class

                #keep count of each word in positive context
                if not (word in freqPositive):
                    freqPositive[word] = 1
                else:
                    freqPositive[word] += 1    
            else:
                negWordsTot+=1# keeps count of total words in negative class
                
                #keep count of each word in positive context
                if not (word in freqNegative):
                    freqNegative[word] = 1
                else:
                    freqNegative[word] += 1

    for word in dictionary:
        #do some smoothing so that minimum count of a word is 1
        if not (word in freqNegative):
            freqNegative[word] = 1
        if not (word in freqPositive):
            freqPositive[word] = 1

        # Calculate p(word|positive)
        pWordPos[word] = freqPositive[word] / float(posWordsTot)

        # Calculate p(word|negative) 
        pWordNeg[word] = freqNegative[word] / float(negWordsTot)

        # Calculate p(word)
        pWord[word] = (freqPositive[word] + freqNegative[word]) / float(allWordsTot) 



In [2]:
def printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred):
    accuracy = correct/total
    precision_pos = correctpos/totalpospred
    precision_neg = correctneg/totalnegpred
    recall_pos = correctpos/totalpos
    recall_neg = correctneg/totalneg
    F1_score_pos = (2*precision_pos*recall_pos)/(precision_pos+recall_pos)
    F1_score_neg = (2*precision_neg*recall_neg)/(precision_neg+recall_neg)
    
    print(dataName)
    print("Accuracy :",accuracy)
    print("Precision - Positive :",precision_pos)
    print("Precision - Negative :",precision_neg)
    print("Recall - Positive :",recall_pos)
    print("Recall - Negative :",recall_neg)
    print("F1 Score - Positive :",F1_score_pos)
    print("F1 Score - Negative :",F1_score_neg,'\n\n')

In [3]:
PRINT_ERRORS=0

#implement naive bayes algorithm
#INPUTS:
#  sentencesTest is a dictonary with sentences associated with sentiment 
#  dataName is a string (used only for printing output)
#  pWordPos is dictionary storing p(word|positive) for each word
#     i.e., pWordPos["apple"] will return a real value for p("apple"|positive)
#  pWordNeg is dictionary storing p(word|negative) for each word
#  pWord is dictionary storing p(word)
#  pPos is a real number containing the fraction of positive reviews in the dataset
def testBayes(sentencesTest, dataName, pWordPos, pWordNeg, pWord,pPos):
    pNeg=1-pPos

    #These variables will store results
    total=0
    correct=0
    totalpos=0
    totalpospred=0
    totalneg=0
    totalnegpred=0
    correctpos=0
    correctneg=0

    #for each sentence, sentiment pair in the dataset
    for sentence, sentiment in sentencesTest.items():
        wordList = re.findall(r"[\w']+", sentence)#collect all words

        pPosW=pPos
        pNegW=pNeg

        for word in wordList: #calculate over unigrams
            if word in pWord:
                if pWord[word]>0.00000001:
                    pPosW *=pWordPos[word]
                    pNegW *=pWordNeg[word]

        prob=0            
        if pPosW+pNegW >0:
            prob=pPosW/float(pPosW+pNegW)


        total+=1
        if sentiment=="positive":
            totalpos+=1
            if prob>0.5:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
                if PRINT_ERRORS:
                    print ("ERROR (pos classed as neg %0.2f):" %prob + sentence)
        else:
            totalneg+=1
            if prob<=0.5:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
                if PRINT_ERRORS:
                    print ("ERROR (neg classed as pos %0.2f):" %prob + sentence)

# TODO for Step 2: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
# (3) precision and recall for the negative class; (4) F1 score;
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)
    

In [4]:

# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
def testDictionary(sentencesTest, dataName, sentimentDictionary, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        for word in Words:
            if word in sentimentDictionary:
               score+=sentimentDictionary[word]
 
        total+=1
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [5]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
def testDictionaryImproved(sentencesTest, dataName, sentimentDictionary, threshold, intensDimiDict, negationList):
    #total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(Words)
        for i in range(0,total):
            if Words[i] in sentimentDictionary:
                score+=4*sentimentDictionary[Words[i]]
                
                ## NEGATION RULES
                if Words[i-1]==negationList:
                   score += -1*score-1

                if sentiment=='positive':
                    ## CAPITALIZATION
                    if Words[i].isupper():
                        score+=1
                    
                    ## Intensifier/Diminisher Rule
                    if Words[i] in intensDimiDict:
                        score+= intensDimiDict[Words[i]]

                    ## Exclamation Rule
                    if Words[i][-1]=='!':
                        score+=1
                
                else:
                    ## CAPITALIZATION
                    if Words[i].isupper():
                        score+=1
                    
                    ## Intensifier/Diminisher Rule
                    if Words[i] in intensDimiDict:
                        score-= intensDimiDict[Words[i]]

                    ## Exclamation Rule
                    if Words[i][-1]=='!':
                        score-=1
                        
 
        #total+=1
        print(score)
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [6]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from afinn import Afinn

def testDictionaryImproved2(sentencesTest, dataName, threshold):
    #total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    afinn = Afinn()
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(Words)
        for word in Words:
            score += afinn.score(word)
                        
 
        #total+=1
        #print(score)
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [7]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from textblob import TextBlob

def testDictionaryImproved3(sentencesTest, dataName, threshold):
    #total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    afinn = Afinn()
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(Words)
        for word in Words:
            word_blob = TextBlob(word)
            score += word_blob.sentiment.polarity
                        
 
        #total+=1
        #print(score)
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [8]:
lexicon_path = "vader_lexicon.txt"

vader_lexicon = {}

with open(lexicon_path, 'r', encoding='utf-8') as file:
    next(file)
    for line in file:
        fields = line.strip().split('\t')
        if len(fields) == 4:
            word = fields[0]
            sentiment = float(fields[1])
            vader_lexicon[word] = sentiment

In [9]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from textblob import TextBlob

def testDictionaryImproved3(sentencesTest, dataName, threshold):
    #total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    afinn = Afinn()
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(Words)
        for word in Words:
            word_blob = TextBlob(word)
            score += word_blob.sentiment.polarity
                        
 
        #total+=1
        #print(score)
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [10]:
#Print out n most useful predictors
def mostUseful(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]<0.0000001:
            predictPower=1000000000
        else:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])
            
    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    print ("NEGATIVE:")
    print (head)
    print ("\nPOSITIVE:")
    print (tail)

In [21]:
sentimentDictionary={} # {} initialises a dictionary [hash function]
sentencesTrain={}
sentencesTest={}
sentencesNokia={}

#initialise datasets and dictionaries
readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia)

pWordPos={} # p(W|Positive)
pWordNeg={} # p(W|Negative)
pWord={}    # p(W) 

#build conditional probabilities using training data
trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord)

In [22]:
#run naive bayes classifier on datasets
print ("Naive Bayes")
testBayes(sentencesTrain,  "Films (Train Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesTest,  "Films  (Test Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesNokia, "Nokia   (All Data,  Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.7)

Naive Bayes
Films (Train Data, Naive Bayes)	
Accuracy : 0.8907029950083195
Precision - Positive : 0.8978551709492462
Precision - Negative : 0.8838394130833503
Recall - Positive : 0.8812005002084202
Recall - Negative : 0.9001660440016604
F1 Score - Positive : 0.8894498790364995
F1 Score - Negative : 0.8919280205655526 


Films  (Test Data, Naive Bayes)	
Accuracy : 0.7717287488061128
Precision - Positive : 0.7773584905660378
Precision - Negative : 0.7659574468085106
Recall - Positive : 0.7729831144465291
Recall - Negative : 0.7704280155642024
F1 Score - Positive : 0.77516462841016
F1 Score - Negative : 0.7681862269641125 


Nokia   (All Data,  Naive Bayes)	
Accuracy : 0.5789473684210527
Precision - Positive : 0.7803030303030303
Precision - Negative : 0.3805970149253731
Recall - Positive : 0.553763440860215
Recall - Negative : 0.6375
F1 Score - Positive : 0.6477987421383647
F1 Score - Negative : 0.4766355140186916 




In [18]:
#run sentiment dictionary based classifier on datasets
testDictionary(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, -4)
testDictionary(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, -4)
testDictionary(sentencesNokia, "Nokia   (All Data, Rule-Based)\t",  sentimentDictionary, -3)

Films (Train Data, Rule-Based)	
Accuracy : 0.544595576783304
Precision - Positive : 0.5380116959064327
Precision - Negative : 0.5526558891454966
Recall - Positive : 0.5955314261850073
Recall - Negative : 0.49421726559273027
F1 Score - Positive : 0.5653121902874132
F1 Score - Negative : 0.5218054949847362 


Films  (Test Data, Rule-Based)	
Accuracy : 0.5455426356589147
Precision - Positive : 0.5619694397283531
Precision - Negative : 0.5237020316027088
Recall - Positive : 0.6107011070110702
Recall - Negative : 0.47346938775510206
F1 Score - Positive : 0.5853227232537578
F1 Score - Negative : 0.49732047159699894 


Nokia   (All Data, Rule-Based)	
Accuracy : 0.6766917293233082
Precision - Positive : 0.7941176470588235
Precision - Negative : 0.46875
Recall - Positive : 0.7258064516129032
Recall - Negative : 0.5625
F1 Score - Positive : 0.7584269662921348
F1 Score - Negative : 0.5113636363636364 




In [ ]:
# print most useful words
mostUseful(pWordPos, pWordNeg, pWord, 100)

In [19]:
#run sentiment dictionary based classifier on datasets
testDictionaryImproved2(sentencesTrain,  "Films (Train Data, Rule-Based)\t", -5)
testDictionaryImproved2(sentencesTest,  "Films  (Test Data, Rule-Based)\t", -10)
testDictionaryImproved2(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", 5)

Films (Train Data, Rule-Based)	
Accuracy : 410.25
Precision - Positive : 0.5043153969099627
Precision - Negative : 0.7723577235772358
Recall - Positive : 0.9883065358112341
Recall - Negative : 0.039239983477901696
F1 Score - Positive : 0.6678425285734443
F1 Score - Negative : 0.07468553459119498 


Films  (Test Data, Rule-Based)	
Accuracy : 33.9375
Precision - Positive : 0.5257032007759457
Precision - Negative : 1.0
Recall - Positive : 1.0
Recall - Negative : 0.0020408163265306124
F1 Score - Positive : 0.6891290527654164
F1 Score - Negative : 0.004073319755600815 


Nokia   (All Data, Rule-Based)	
Accuracy : 14.625
Precision - Positive : 0.9743589743589743
Precision - Negative : 0.34801762114537443
Recall - Positive : 0.20430107526881722
Recall - Negative : 0.9875
F1 Score - Positive : 0.33777777777777784
F1 Score - Negative : 0.5146579804560261 




In [20]:
#run sentiment dictionary based classifier on datasets
testDictionaryImproved3(sentencesTrain,  "Films (Train Data, Rule-Based)\t", 0)
testDictionaryImproved3(sentencesTest,  "Films  (Test Data, Rule-Based)\t", -2)
testDictionaryImproved3(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", 0)

Films (Train Data, Rule-Based)	
Accuracy : 492.0833333333333
Precision - Positive : 0.5748275376601436
Precision - Negative : 0.7207278481012658
Recall - Positive : 0.852578826477344
Recall - Negative : 0.37629078893019413
F1 Score - Positive : 0.6866801210898082
F1 Score - Negative : 0.49443690637720494 


Films  (Test Data, Rule-Based)	
Accuracy : 34.0625
Precision - Positive : 0.5267249757045676
Precision - Negative : 1.0
Recall - Positive : 1.0
Recall - Negative : 0.006122448979591836
F1 Score - Positive : 0.6900063653723743
F1 Score - Negative : 0.012170385395537523 


Nokia   (All Data, Rule-Based)	
Accuracy : 25.625
Precision - Positive : 0.7777777777777778
Precision - Negative : 0.7317073170731707
Recall - Positive : 0.9408602150537635
Recall - Negative : 0.375
F1 Score - Positive : 0.8515815085158152
F1 Score - Negative : 0.4958677685950414 


